In [4]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_census_income
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=pre_census_income.X_train.shape[1:]),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            'r': [6],
            'g': [7],
            'a&r': [0, 6],
            'a&g': [0, 7],
            'r&g': [6, 7]
            }

model_path = "models/adult_model.h5"
# adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
model = keras.models.load_model(model_path)
model.summary()

# similar_X = [dis_data, new_data]

layer_map = [('layer1', 'scale_layer_5'), ('layer2', 'scale_layer_6'), ('layer3', 'scale_layer_7'), ('layer4', 'scale_layer_8'), ('layer5', 'scale_layer_8'), ('layer6', 'scale_layer_8')]

from preprocessing import pre_census_income
from tensorflow.keras.models import Model

for layer_index in range(4, 6):
    print("layer", layer_index+1)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_census_income.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_census_income.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][0]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_census_income.X_train)

        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()
        
        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
layer4 (Dense)               (None, 15)                240       
_________________________________________________________________
layer5 (Dense)               (None, 10)                160       
_________________________________________________________________
layer6 (Dense)               (None, 1)                 11        
Total params: 1,736
Trainable params: 1,736
Non-trainable params: 0
______________________________________________________

In [3]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_bank_marketing
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=pre_bank_marketing.X_train.shape[1:]),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            }

model_path = "models/bank_model.h5"
# adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
model = keras.models.load_model(model_path)
model.summary()

# similar_X = [dis_data, new_data]

layer_map = [('layer1', 'scale_layer_5'), ('layer2', 'scale_layer_6'), ('layer3', 'scale_layer_7'), ('layer4', 'scale_layer_8'), ('layer5', 'scale_layer_8'), ('dense_137', 'scale_layer_8')]

from preprocessing import pre_bank_marketing
from tensorflow.keras.models import Model

for layer_index in range(4,6):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_bank_marketing.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_bank_marketing.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][0]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_bank_marketing.X_train)
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 30)                510       
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
layer4 (Dense)               (None, 15)                240       
_________________________________________________________________
layer5 (Dense)               (None, 10)                160       
_________________________________________________________________
dense_137 (Dense)            (None, 1)                 11        
Total params: 1,856
Train

In [5]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_census_income
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=pre_census_income.X_train.shape[1:]),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            'r': [6],
            'g': [7],
            'a&r': [0, 6],
            'a&g': [0, 7],
            'r&g': [6, 7]
            }

model_path = "models/retrained_model_EIDIG/adult_EIDIG_INF_retrained_model.h5"
# adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
model = keras.models.load_model(model_path)
model.summary()

# similar_X = [dis_data, new_data]

layer_map = [('layer1', 'scale_layer_5'), ('layer2', 'scale_layer_6'), ('layer3', 'scale_layer_7'), ('layer4', 'scale_layer_8'), ('layer5', 'scale_layer_8'), ('dense_5', 'scale_layer_8')]

from preprocessing import pre_census_income
from tensorflow.keras.models import Model

for layer_index in range(4, 6):
    print("layer", layer_index+1)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_census_income.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_census_income.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][0]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_census_income.X_train)
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))
                                 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 30)                390       
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
layer4 (Dense)               (None, 15)                240       
_________________________________________________________________
layer5 (Dense)               (None, 10)                160       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 1,736
Trainable params: 1,736
Non-trainable params: 0
______________________________________________________

In [7]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_bank_marketing
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=pre_bank_marketing.X_train.shape[1:]),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 'a': [0],
            }

model_path = "models/retrained_model_EIDIG/bank_EIDIG_INF_retrained_model.h5"
# adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
model = keras.models.load_model(model_path)
model.summary()

# similar_X = [dis_data, new_data]

layer_map = [("layer1", 'scale_layer_5'), ('layer2', 'scale_layer_6'), ('layer3', 'scale_layer_7'), ('layer4', 'scale_layer_8'), ('layer5', 'scale_layer_8'), ('dense_17', 'scale_layer_8')]

from preprocessing import pre_bank_marketing
from tensorflow.keras.models import Model

for layer_index in range(5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_bank_marketing.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_bank_marketing.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][0]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_bank_marketing.X_train)
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 30)                510       
_________________________________________________________________
layer2 (Dense)               (None, 20)                620       
_________________________________________________________________
layer3 (Dense)               (None, 15)                315       
_________________________________________________________________
layer4 (Dense)               (None, 10)                160       
_________________________________________________________________
layer5 (Dense)               (None, 5)                 55        
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 6         
Total params: 1,666
Trainable params: 1,666
Non-trainable params: 0
____________________________________________________

In [8]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_lsac
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(50, activation="relu", input_shape=pre_lsac.X_train.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(5, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(int(constraint[i][0]), int(constraint[i][1]+1)))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 
            'r': [10],
            'g': [9],
            'g&r': [10, 9]
            }

model_path = "models/lsac_model.h5"
# adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
model = keras.models.load_model(model_path)
model.summary()

# similar_X = [dis_data, new_data]

layer_map = [('layer1', 'scale_layer_5'), ('layer2', 'scale_layer_6'), ('layer3', 'scale_layer_7'), ('layer4', 'scale_layer_8'), ('layer5', 'scale_layer_8')]

from preprocessing import pre_lsac
from tensorflow.keras.models import Model

for layer_index in range(5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_lsac.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_lsac.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][0]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_lsac.X_train)
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
layer4 (Dense)               (None, 10)                160       
_________________________________________________________________
layer5 (Dense)               (None, 5)                 55        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 2,816
Trainable params: 2,816
Non-trainable params: 0
______________________________________________________

In [9]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_lsac
import tensorflow.keras.backend as K
import argparse

seed(1)
tf.random.set_random_seed(2)
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True 
sess = tf.Session(config=config)

K.set_session(sess)

def my_loss_fun(y_true, y_pred):
    # do whatever you want
    return y_pred

    
model = keras.Sequential([
    keras.layers.Dense(50, activation="relu", input_shape=pre_lsac.X_train.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(15, activation="relu"),
    keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(5, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(int(constraint[i][0]), int(constraint[i][1]+1)))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X

pos_map = { 
            'r': [10],
            'g': [9],
            'g&r': [10, 9]
            }

model_path = "models/retrained_model_EIDIG/lsac_EIDIG_INF_retrained_model.h5"
# adult_g_gated_4_0.3_0.2_p-0.1_p0.9.h5
model = tf.keras.models.load_model(model_path, compile=False)
model.summary()

# similar_X = [dis_data, new_data]

layer_map = [('layer1', 'scale_layer_5'), ('layer2', 'scale_layer_6'), ('layer3', 'scale_layer_7'), ('layer4', 'scale_layer_8'), ('layer5', 'scale_layer_8')]

from preprocessing import pre_lsac
from tensorflow.keras.models import Model

for layer_index in range(5):
    print("layer", layer_index)
    for attr in pos_map.keys():
        protected_attribs = pos_map[attr]

        # data_name = f"data/adult/C-{attr}_ids_EIDIG_INF.npy"
        # dis_data = np.load(data_name)

        dis_data = pre_lsac.X_train
        num_attribs = len(dis_data[0])
        new_data = dis_data.copy()

        similar_X = similar_set(dis_data, num_attribs, protected_attribs, pre_lsac.constraint)

        # layer_name = layer_map[layer_index][1]
        # inter_model = Model(model.input, model.get_layer(layer_name).output)

        layer_name = layer_map[layer_index][0]
        inter_model = Model(model.input, model.get_layer(layer_name).output)                                 

        inter_output_ori = inter_model.predict(pre_lsac.X_train)
        max_v = inter_output_ori.max()
        min_v = inter_output_ori.min()

        diff = 0
        for i in range(len(similar_X)):
            inter_output_adv = inter_model.predict(similar_X[i])
            diff += np.abs(inter_output_adv - inter_output_ori).sum() / (max_v - min_v)

        num = len(similar_X) * similar_X[0].shape[0]
        print("attr:", attr, round(diff/num, 3))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 50)                600       
_________________________________________________________________
layer2 (Dense)               (None, 30)                1530      
_________________________________________________________________
layer3 (Dense)               (None, 15)                465       
_________________________________________________________________
layer4 (Dense)               (None, 10)                160       
_________________________________________________________________
layer5 (Dense)               (None, 5)                 55        
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 6         
Total params: 2,816
Trainable params: 2,816
Non-trainable params: 0
____________________________________________________